In [ ]:
import json
import re
import time
import torch
import gc
import numpy as np
from tqdm import tqdm
from pathlib import Path
from typing import Dict, List, Tuple
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
import faiss
from sentence_transformers import SentenceTransformer
from method_db_helper import MethodDatabaseBuilder

# =============================================================================
# CONFIGURATION
# =============================================================================
MODEL_ID = "ytu-ce-cosmos/Turkish-Gemma-9b-T1"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

INPUT_FILE = Path("/kaggle/input/rag-questions/selected_200_questions.json")
OUTPUT_BASE_DIR = Path("method_databases")
OUTPUT_BASE_DIR.mkdir(parents=True, exist_ok=True)

MAX_NEW_TOKENS = 4096
DTYPE = torch.bfloat16

# =============================================================================
# PROMPT TEMPLATES
# =============================================================================
PROMPTS = {  
    "metacognitive": """Analyze the following mathematical problem and its solution.

Problem: {question}

Solution: {solution}

Task: Describe the thinking process and problem-solving strategy. Focus on "what to think about" rather than "what to calculate".

Provide your answer as a JSON object with this exact format:
{{"method": "Description of the thinking process and strategy (4-6 sentences)"}}"""
}

# =============================================================================
# MAIN EXECUTION
# =============================================================================
def main():
    print("\n" + "="*80)
    print("METHOD DATABASE BUILDER")
    print("="*80)
    
    # Load questions
    print(f"\n📥 Loading questions from: {INPUT_FILE}")
    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        questions_data = json.load(f)
    
    print(f"✅ Loaded {len(questions_data)} questions")
    
    # Initialize builder
    builder = MethodDatabaseBuilder(MODEL_ID, EMBEDDING_MODEL)
    
    # Build database for each prompt
    for prompt_name, prompt_template in PROMPTS.items():
        try:
            # Build
            index, metadata = builder.build_database(
                prompt_name, 
                prompt_template, 
                questions_data
            )
            
            # Save
            builder.save_database(
                prompt_name, 
                index, 
                metadata, 
                OUTPUT_BASE_DIR
            )
            
            # Cleanup
            torch.cuda.empty_cache()
            gc.collect()
            
        except Exception as e:
            print(f"\n❌ Error building database for {prompt_name}: {e}")
            continue
    
    print("\n" + "="*80)
    print("✅ ALL DATABASES BUILT SUCCESSFULLY")
    print(f"📂 Output directory: {OUTPUT_BASE_DIR}")
    print("="*80)
    
    # Final cleanup
    del builder.model
    torch.cuda.empty_cache()
    gc.collect()


if __name__ == "__main__":
    main()

2026-01-13 06:29:44.380478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768285784.572022      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768285784.631638      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768285785.086341      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768285785.086377      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768285785.086380      55 computation_placer.cc:177] computation placer alr


METHOD DATABASE BUILDER

📥 Loading questions from: /kaggle/input/rag-questions/selected_200_questions.json
✅ Loaded 200 questions
🔧 Initializing Method Database Builder...

📥 Loading reasoning model: ytu-ce-cosmos/Turkish-Gemma-9b-T1


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

...


Example 5:
Solution: How many trees did Amiyah cut? ** The number of trees that Amiyah cut is 20/100*400 = <<20/100*400=80>>80
How many trees are left on the farm? ** The number of trees remaining on the farm after she cut 80 trees is 400-80 = <<400-80=320>>320
How many new trees did Amiyah plant? ** For every tree that Amiyah cut, she plants 5 new trees, and since she cut 80 trees, she planted 80*5 = <<80*5=400>>400 new trees.
What is the total number of trees on the farm? ** The total number of trees on the farm is now 320+400 = <<320+400=720>>720 trees.
#### 720
Method: Öncelikle, kesilen ağaç sayısını hesaplamak için başlangıçtaki ağaç sayısını ve kesilen yüzdesini kullanırım. Kesilen ağaç sayısı bulunduktan sonra, kesim işlemi sonrası çiftlikte kalan ağaç sayısını hesaplarım. Daha sonra, kesilen her ağaç için dikildiği belirtilen yeni ağaç sayısını hesaplarım. Son olarak, kesim sonrası kalan ağaçlar ile dikilen yeni ağaçların toplamını alarak çiftlikteki son toplam ağaç sayısını 

Generating methods (metacognitive): 100%|██████████| 200/200 [6:11:27<00:00, 111.44s/it]  


🔢 Creating embeddings for 200 methods...


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

🔍 Building FAISS index...
✅ Database built: 200 vectors indexed
💾 Database saved to: method_databases/metacognitive
   - FAISS index: method_databases/metacognitive/faiss_index.bin
   - Metadata: method_databases/metacognitive/metadata.json
   - Summary: method_databases/metacognitive/summary.json

✅ ALL DATABASES BUILT SUCCESSFULLY
📂 Output directory: method_databases
